In [1]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

In [3]:
!ls ..

configs				 sub_100_275_0909_all_1.csv
mmdetection			 sub_20_100_0909_5000_1.csv
notebooks			 sub_20_100_0909_all_1.csv
preds_0902_3_50_all_lb04195.pkl  sub_htc275_0912am_all.csv
scripts				 sub_htc_parent_0912pm_all_1.csv
seg				 sub_parent_0905_1.csv
sub_0_275_0912am_5000_1.csv	 sub_parent_0911_5000_1.csv
sub_100_275_0909_5000_1.csv	 sub_parent_0912am_5000_1.csv
sub_100_275_0909_5000_iou03.csv  work_dirs


In [4]:
csv_files = [
            '../sub_htc275_0912am_all.csv',
            '../sub_htc_parent_0912pm_all_1.csv' #,
            #'sub_0902_3_top50.csv.zip',
            #'sub_parent_0903_1_top50.csv'
            ]

dfs = [pd.read_csv(x) for x in csv_files]

In [5]:
for df in dfs:
    df.PredictionString = df.PredictionString.fillna('')

In [6]:
len(dfs[1].iloc[0].PredictionString)

3438

In [7]:
# lower conf for is model, not working
# dfs[1].PredictionString = dfs[1].PredictionString.map(lambda x: get_low_conf_pred_str(x))

In [8]:
dfs[0].head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/083wq 0.0020746 eNqFU01vwyAM/UsYQtJO3WlVlZS...
1,52a76f3fe21185bf,1024,683,/m/0fszt 0.0313353 eNq9T00PgjAM/UstG3qah8WL7su...
2,65e9da7c04c8e4cd,1024,682,/m/025nd 0.0010005 eNoLDYlIMrDJTjEw8zH2NfIz9jH...
3,6d10fa21d93bd6cc,1024,682,/m/05r655 0.0119960 eNrtU9tugzAM/aVQLiaV9rCHLR...
4,0841dee573a875ca,1024,683,/m/05r655 0.0080835 eNpt0dtOAjEQBuBX6rQQoy0anB...


In [9]:
#keep top 100
#dfs[0].PredictionString = dfs[0].PredictionString.map(lambda x: ' '.join(x.split(' ')[:1200]))

In [10]:
dfs[1].head(10)

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/0k4j 0.0144516 eNqFU7luwzAM/SWdcbK0yCZbBxEP...
1,52a76f3fe21185bf,1024,683,/m/01g317 0.0056462 eNozzTZKM4gyzLdJ8wt0xk4HOE...
2,65e9da7c04c8e4cd,1024,682,/m/015p6 0.0019611 eNqLSgswNw1NNTD0B0IDA38jP0M...
3,6d10fa21d93bd6cc,1024,682,/m/01g317 0.4327452 eNrNUttugzAM/aWGQAOttIdJBb...
4,0841dee573a875ca,1024,683,/m/0k4j 0.0063638 eNoLCU0yMg1MNbBxM/c08zT3NvYx...
5,660329181edd20b4,1024,483,/m/01g317 0.0118703 eNpNUWFPxCAM/UuFjrGdyRKj2c...
6,2dffa0787ba35f70,1024,545,/m/0c9ph5 0.9175597 eNpVT00LwjAM/Ut5bUXGDkM8OW...
7,cb66703ecbd0b99a,1024,678,/m/0k4j 0.0013056 eNoLTzBIM0hIMQACf0MgYehvgAOA...
8,2f01f2c68316b5e5,1024,768,/m/0k4j 0.1164669 eNoLC01OMjDMSzcw8jP0NzQwAGJD...
9,36cfe3fa46f56ed3,768,768,/m/0k4j 0.0147882 eNrLzQk1MspKNzDzNHez8jHzNPU1...


In [11]:
x = None
for d in dfs:
    if x is None:
        x = d.PredictionString
    else:
        x = x.str.cat(d.PredictionString, sep=' ')

In [12]:
dfs[0].PredictionString = x
print(len(dfs[0].iloc[0].PredictionString), len(dfs[1].iloc[0].PredictionString))

23927 3438


In [13]:
dfs[0].head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/083wq 0.0020746 eNqFU01vwyAM/UsYQtJO3WlVlZS...
1,52a76f3fe21185bf,1024,683,/m/0fszt 0.0313353 eNq9T00PgjAM/UstG3qah8WL7su...
2,65e9da7c04c8e4cd,1024,682,/m/025nd 0.0010005 eNoLDYlIMrDJTjEw8zH2NfIz9jH...
3,6d10fa21d93bd6cc,1024,682,/m/05r655 0.0119960 eNrtU9tugzAM/aVQLiaV9rCHLR...
4,0841dee573a875ca,1024,683,/m/05r655 0.0080835 eNpt0dtOAjEQBuBX6rQQoy0anB...


In [14]:
dfs[0].to_csv('merge_0912_1.csv', index=False)

In [15]:
!kaggle competitions submit -c open-images-2019-instance-segmentation -f merge_0912_1.csv -m "merge 0912"

100%|██████████████████████████████████████| 2.55G/2.55G [02:07<00:00, 21.5MB/s]
Successfully submitted to Open Images 2019 - Instance Segmentation

In [ ]:
!ls

In [11]:
len(dfs[0].loc[dfs[0].PredictionString.isna()])

10336

In [56]:
len(dfs[0].loc[dfs[0].PredictionString.isna()])

0

In [12]:
len(dfs[1].loc[dfs[1].PredictionString.isna()])

41

In [100]:
#####
lst = ['Geeks', 'For', 'Geeks', 'is',  
            'portal', 'for', 'Geeks'] 
tmp1 = pd.DataFrame(lst, columns=['t'])
lst2 = ['a', 'b', 'c', 'd',  
            'e', 'f', 'g'] 
tmp2 = pd.DataFrame(lst2, columns=['t'])

lst3 = ['a1', 'b1', 'c', 'd',  
            'e', 'f', 'g'] 
tmp3 = pd.DataFrame(lst3, columns=['t'])
tmp3.head()

,t
0,a1
1,b1
2,c
3,d
4,e


In [4]:
df_is = pd.read_csv('ensemble_is_42_0808_1_3_models.csv')
df_is.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.1689330 /m/04bcr3 0.3664692 0.7250217 0.4814...
1,763c72626bf24534,0.7671249 /m/04bcr3 0.6873273 0.8193369 0.9977...
2,7540c162180973b1,0.0219995 /m/04dr76w 0.0762036 0.3234025 0.120...
3,c4e522dce52cb02f,0.4940703 /m/04bcr3 0.8307936 0.6809913 0.9920...
4,cafcd9496c808085,0.0129044 /m/04dr76w 0.0000000 0.0600080 0.031...


In [5]:
def get_low_conf_pred_str(pred_str):
    try:
        if len(pred_str) < 1:
            return pred_str
    except:
        print('pred_str:', pred_str)
        raise
    det_items = pred_str.split(' ')
    
    res = []
    for i, c in enumerate(det_items):
        if i % 12 == 0:
            res.append(str(float(c)/4))
        else:
            res.append(c)
    return ' '.join(res)